In [19]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
cheesecake_train_dir = '../data/food11/train/cheesecake'
cheesecake_test_dir = '../data/food11/test/cheesecake'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(cheesecake_train_dir, fname) for fname in os.listdir(cheesecake_train_dir)]
image_paths += [os.path.join(cheesecake_test_dir, fname) for fname in os.listdir(cheesecake_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 999 images


In [20]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 10

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/cheesecake\1001446.jpg assigned to cluster 1
../data/food11/train/cheesecake\1004515.jpg assigned to cluster 7
../data/food11/train/cheesecake\1004807.jpg assigned to cluster 3
../data/food11/train/cheesecake\1017408.jpg assigned to cluster 0
../data/food11/train/cheesecake\1021942.jpg assigned to cluster 2
../data/food11/train/cheesecake\1023543.jpg assigned to cluster 5
../data/food11/train/cheesecake\1034351.jpg assigned to cluster 1
../data/food11/train/cheesecake\1035453.jpg assigned to cluster 2
../data/food11/train/cheesecake\1037609.jpg assigned to cluster 6
../data/food11/train/cheesecake\1047482.jpg assigned to cluster 8
../data/food11/train/cheesecake\1055684.jpg assigned to cluster 2
../data/food11/train/cheesecake\1057054.jpg assigned to cluster 3
../data/food11/train/cheesecake\1061232.jpg assigned to cluster 2
../data/food11/train/cheesecake\1061989.jpg assigned to cluster 1
../data/food11/train/cheesecake\1073148.jpg assigned to cluster 9
../data/fo

In [21]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(cheesecake_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(cheesecake_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(cheesecake_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(cheesecake_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [22]:
# Fine-grained apple_pie classification
cheesecake_train_gen = ImageDataGenerator(rescale=1./255)
cheesecake_test_gen = ImageDataGenerator(rescale=1./255)

cheesecake_train_generator = cheesecake_train_gen.flow_from_directory(
    cheesecake_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

cheesecake_test_generator = cheesecake_test_gen.flow_from_directory(
    cheesecake_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_cheesecake = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_cheesecake.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_cheesecake = base_model_cheesecake.output
x_cheesecake = GlobalAveragePooling2D()(x_cheesecake)
x_cheesecake = Dense(512, activation='relu')(x_cheesecake)
cheesecake_predictions = Dense(num_clusters, activation='softmax')(x_cheesecake)

# Create and compile the model
cheesecake_model = Model(inputs=base_model_cheesecake.input, outputs=cheesecake_predictions)
cheesecake_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
cheesecake_model.fit(
    cheesecake_train_generator,
    steps_per_epoch=cheesecake_train_generator.samples // 32,
    validation_data=cheesecake_test_generator,
    validation_steps=cheesecake_test_generator.samples // 32,
    epochs=30
)

# Save the model
cheesecake_model.save('fine_grained_cheesecake_classifier.h5')

Found 900 images belonging to 10 classes.
Found 99 images belonging to 10 classes.
Epoch 1/30
28/28 [==============================] - 9s 292ms/step - loss: 2.1448 - accuracy: 0.2500 - val_loss: 1.6627 - val_accuracy: 0.5625
Epoch 2/30
28/28 [==============================] - 7s 265ms/step - loss: 1.3673 - accuracy: 0.5922 - val_loss: 1.2082 - val_accuracy: 0.7188
Epoch 3/30
28/28 [==============================] - 7s 258ms/step - loss: 0.9698 - accuracy: 0.7523 - val_loss: 1.0107 - val_accuracy: 0.7083
Epoch 4/30
28/28 [==============================] - 7s 257ms/step - loss: 0.7284 - accuracy: 0.8364 - val_loss: 0.8370 - val_accuracy: 0.7500
Epoch 5/30
28/28 [==============================] - 7s 259ms/step - loss: 0.5642 - accuracy: 0.8940 - val_loss: 0.7442 - val_accuracy: 0.8125
Epoch 6/30
28/28 [==============================] - 8s 278ms/step - loss: 0.4464 - accuracy: 0.9309 - val_loss: 0.7167 - val_accuracy: 0.7708
Epoch 7/30
28/28 [==============================] - 7s 260ms/step